In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from joblib import dump,load
from sklearn.feature_selection import RFECV

In [2]:
#读数据标准化划分数据集
df = pd.read_csv('../my_data.csv')
X = df.iloc[:, 1:58]
Y = df.iloc[:, 58:]
X_to_select = ['A+B_Ionic_radius', 'B+X_Ionic_radius', 'A/X_Ionic_radius',
               'B/X_Ionic_radius', 'Tolerance factor', 'Octahedral factor', 
               'B_Electron_affinity(kJ/mol)', 'B_Valence_electrons', 'B_Heat_of_vap(kJ/mol)',
               'B_Electronegativity']
X=X[X_to_select]
Y=Y['SLME @ 5um']
names=df.columns
names=['SHAP value of '+x for x in names]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)
print("X_train's shape is", X_train.shape,"; y_train's shape is", y_train.shape)
print("X_test's shape is", X_test.shape,"; y_test's shape is",y_test.shape)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_stand = scaler.transform(X_train)
X_test_stand = scaler.transform(X_test)

X_train's shape is (390, 10) ; y_train's shape is (390,)
X_test's shape is (98, 10) ; y_test's shape is (98,)


In [3]:
##gridsearch and cv select model##
max_depth = [int(x) for x in np.linspace(10, 500, num =10)]
max_depth.append(None)
param_grid = {
    'n_estimators': [10, 20, 30, 40, 50, 100, 200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': max_depth,
    'min_samples_split': [1, 2, 3, 4, 5],
    'min_samples_leaf': [1, 2, 3, 4],
    'bootstrap': [True, False]
}

In [4]:
rf = RandomForestRegressor()
model = GridSearchCV(estimator =rf,scoring='neg_root_mean_squared_error', 
                     param_grid = param_grid,cv=3, verbose=3,n_jobs =-1,return_train_score=True)
model.fit(X_train, y_train)
print(model.cv_results_)
dfresult=pd.DataFrame(model.cv_results_)
dfresult.to_csv('SLME_RF13y32.csv')
print(dfresult)
print(model.best_params_)
print(model.scorer_)
print(model.best_score_)
bestmodel=model.best_estimator_

Fitting 3 folds for each of 10560 candidates, totalling 31680 fits


D:\App\Anaconda3\envs\test\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
6336 fits failed out of a total of 31680.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3168 fits failed with the following error:
Traceback (most recent call last):
  File "D:\App\Anaconda3\envs\test\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\App\Anaconda3\envs\test\lib\site-packages\sklearn\ensemble\_forest.py", line 467, in fit
    for i, t in enumerate(trees)
  File "D:\App\Anaconda3\envs\test\lib\site-packages\joblib\parallel.py", line 1048, in __call__
    if self.dispatch_one_batch(iterator):
  File "D:\App\Anaconda3\en

{'mean_fit_time': array([0.00635282, 0.0096906 , 0.01136374, ..., 0.09636378, 0.17684929,
       0.37084047]), 'std_fit_time': array([0.00085442, 0.00085112, 0.00023768, ..., 0.00155286, 0.00071593,
       0.00674438]), 'mean_score_time': array([0.        , 0.        , 0.        , ..., 0.00885057, 0.01085107,
       0.02171532]), 'std_score_time': array([0.        , 0.        , 0.        , ..., 0.00047154, 0.00047092,
       0.00065908]), 'param_bootstrap': masked_array(data=[True, True, True, ..., False, False, False],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[10, 10, 10, ..., None, None, None],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_max_features': masked_array(data=['auto', 'auto', 'auto', ..., 'log2', 'log2', 'log2'],
             mask=[False, False, False, ..., False, False, False],
    